In [32]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras.callbacks import CSVLogger

In [ ]:
from src.model_fit_funcs import load_image, make_ds, build_model, finetune

In [2]:
model_test, base_test = build_model()
model_test.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ included_input (InputLayer)     │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenet_1.00_224 (Functional) │ (None, 7, 7, 1024)     │     3,228,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ GAP_Head                        │ (None, 1024)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Regressor (Dense)               │ (None, 1)              │         1,025 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,229,889 (12.32 MB)

 Trainable params: 1,025 (4.00 KB)

 Non-trainable params: 3,228,864 (12.32 MB)

In [3]:
finetune(model_test, base_test)
model_test.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ included_input (InputLayer)     │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenet_1.00_224 (Functional) │ (None, 7, 7, 1024)     │     3,228,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ GAP_Head                        │ (None, 1024)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Regressor (Dense)               │ (None, 1)              │         1,025 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,229,889 (12.32 MB)

 Trainable params: 1,854,465 (7.07 MB)

 Non-trainable params: 1,375,424 (5.25 MB)

# Carregando o arquivo com os rankings dos hiperparâmetros

In [13]:
best_score = pd.read_csv('reports\\random_search_20251023_202559.csv')

In [14]:
best_score.head(5)

,trial,lr_head,lr_ft,ep_head,ep_ft,n_unfreeze,r2
0,15,0.0001,0.000300,15,10,15,0.905133
1,8,0.0001,0.000300,10,10,15,0.728941
2,1,0.0001,0.000300,20,10,8,0.704817
3,4,0.0030,0.000001,20,10,5,0.701162
4,13,0.0030,0.000300,15,20,8,0.657441


In [15]:
best_score.iloc[0]['trial']

np.float64(15.0)

In [17]:
b_lr_head = float(best_score.iloc[0]['lr_head'])
b_lr_ft = float(best_score.iloc[0]['lr_ft'])
b_ep_head = float(best_score.iloc[0]['ep_head'])
b_ep_ft = float(best_score.iloc[0]['ep_ft'])
b_n_unfreeze = float(best_score.iloc[0]['n_unfreeze'])

# Criação do Dataset

In [16]:
df = pd.read_csv('resultados.csv')

In [19]:
paths_imagens = df['file_path']
y = df['Resultados']

In [21]:
ds = make_ds(paths_imagens, y, batch_size=8)

# Ajuste com todo o dataset

In [33]:
cb = [
    keras.callbacks.EarlyStopping(monitor='loss', patience=3, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=2, min_lr=1e-6), # Monitoramento por loss, sem validação
]

In [ ]:
# Criação do backbone
model, base = build_model(b_lr_head)

model.fit(ds, epochs=b_ep_head, callbacks=cb, verbose=0)


In [ ]:
# Fine-tuning

finetune(model, base, lr=b_lr_ft, n_unfreeze=b_n_unfreeze)
model.fit(ds, epochs=b_ep_ft,  callbacks=[cb], verbose=0)

In [ ]:
model.save('models\\modelo_final.keras')

In [28]:
from datetime import datetime, timedelta
from time import sleep

In [29]:
começo = datetime.now()

duração = timedelta(minutes = 1)

agora = datetime.now()

while agora - começo < duração:
    sleep(2)
    print('ainda não')
    agora = datetime.now()
print('agora sim')

ainda não
ainda não
ainda não
ainda não
ainda não
ainda não
ainda não
ainda não
ainda não
ainda não
ainda não
ainda não
ainda não
ainda não
ainda não
ainda não
ainda não
ainda não
ainda não
ainda não
ainda não
ainda não
ainda não
ainda não
ainda não
ainda não
ainda não
ainda não
ainda não
ainda não
agora sim


In [31]:
agora - começo

datetime.timedelta(seconds=60, microseconds=75995)